In [ ]:
#!/usr/bin/env python
# coding: utf-8
#######################################################################
# How to setup AndroSensor:
# Install AndroSensor
# Ensure when installing, allow app to run in background.
# Open settings and change recording interval to fast.
#
#
#
#######################################################################
import glob
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'
import numpy as np
import scipy as sp
import scipy.signal as sg

px.set_mapbox_access_token(open(".mapbox_token").read())

In [ ]:
# IF useing ; sep, then delete the first line.
path = r'phone_data'  # use your path
all_files = glob.glob(os.path.join(path, "*.csv")) 

df_from_each_file = [pd.read_csv(f, sep=";") for f in all_files]
concatenated_df = pd.concat(df_from_each_file, ignore_index=True, sort=True)

# Convert datatime to actual datetime
concatenated_df["timestamp"] = pd.to_datetime(concatenated_df["YYYY-MO-DD HH-MI-SS_SSS"],format='%Y-%m-%d %H:%M:%S:%f')
concatenated_df["Speed(km/h)"] = concatenated_df["LOCATION Speed ( Kmh)"]

In [ ]:
# calculate abs accleration
concatenated_df["abs_acceleration"] = np.linalg.norm(
    concatenated_df[["ACCELEROMETER X (m/s²)", "ACCELEROMETER Y (m/s²)", "ACCELEROMETER Z (m/s²)"]].values, axis=1)

# Calculate abs location
concatenated_df["abs_loc"] = np.linalg.norm(concatenated_df[['LOCATION Latitude : ', 'LOCATION Longitude : ']].values,
                                            axis=1)

# Parse GPS sats properly
concatenated_df['Satellites in range'] = concatenated_df['Satellites in range'].str.extract(r'(\d*) \/ \d*')
concatenated_df['Satellites in range'] = pd.to_numeric(concatenated_df['Satellites in range'] )

In [ ]:
concatenated_df = concatenated_df[~(concatenated_df['timestamp'] < '2021-03-12 18:46:00')]
concatenated_df = concatenated_df[~(concatenated_df['timestamp'] > '2021-03-12 18:52:00')]

# Display 3-axis acceleration vs time

In [ ]:
# Display accelration
fig = px.line(concatenated_df,
              x="timestamp",
              y=["ACCELEROMETER X (m/s²)", "ACCELEROMETER Y (m/s²)", "ACCELEROMETER Z (m/s²)"],
              title='Acceleration'
             )


In [ ]:
fig.show()

# Now process data from display GPS positions

In [ ]:
# remove all duplicated long lat positions
unique_coords_df = concatenated_df.groupby('abs_loc').last().reset_index()
unique_coords_df = unique_coords_df.sort_values(by='timestamp')

In [ ]:
unique_coords_df["gps_acceleration"] = unique_coords_df["LOCATION Speed ( Kmh)"].diff()
unique_coords_df["power"] = unique_coords_df["gps_acceleration"] * unique_coords_df["LOCATION Speed ( Kmh)"] * 80

#unique_coords_df.to_csv(
#    r"D:\OneDrive - Imperial College London\University Storage\Masters project\Raw data\combined.csv")

In [ ]:
unique_coords_df["slope"] = unique_coords_df["LOCATION Altitude ( m)"].diff()
#unique_coords_df["slope"].describe()

In [ ]:
#unique_coords_df["slope"].mask(unique_coords_df["slope"].between(-200, -10), inplace=True)
#unique_coords_df["slope"].mask(unique_coords_df["slope"].between(10, 200), inplace=True)

In [ ]:
unique_coords_df.dropna(subset = ['Speed(km/h)'], inplace = True) 

# Parse data from ebike datalogger

In [ ]:
my_cols = range(16)
df = pd.read_csv("raw_data/data_19-4-21.csv",
            names=my_cols,
            engine='c')

In [ ]:
df.head()

In [ ]:
df['Datetime'] = pd.to_datetime(df[0])
df.sort_values(by='Datetime',inplace = True)

df.rename(columns={0: 'datetime_string',
                       1: 'sensor',
                      }, inplace=True)

In [ ]:
df = df[~(df['Datetime'] < '2020-03-12 18:46:00')]

In [ ]:
df.dtypes

In [ ]:
mask = df["sensor"] == 'gps'
df_gps = df[mask]

df_gps.rename(columns={2: 'hour',
                       3: 'minute',
                       4: 'second',
                       5: 'millisecond',
                       6: 'latitude',
                       7: 'longitude',
                       8: 'altitude',
                       9: 'GPS Speed [km/h]',
                       10: 'sats',
                       11: 'gnssFixOK',
                       12: 'fix_type',
                      }, inplace=True)
def insert_time(row):
    return row['Datetime'].replace(minute=int(row['minute']),second=int(row['second']),microsecond=int(row['millisecond']*1000))

df_gps['Datetime'] = df_gps.apply(lambda r: insert_time(r), axis=1)
df_gps.sort_values(by='Datetime',inplace = True)

time_delta = df_gps["Datetime"].diff().dt.total_seconds().fillna(0)
df_gps['acceleration'] = df_gps["GPS Speed [km/h]"].diff()/time_delta

df_gps.dropna(axis=1, how='all',inplace=True)
df_gps.head()

In [ ]:
mask = df["sensor"] == 'pas'
df_pas = df[mask]

N_PAS_MAGNETS = 12

df_pas.rename(columns={2: 'pulse_delay_us',
                      }, inplace=True)
df_pas.dropna(axis=1, how='all',inplace=True)
df_pas = df_pas[df_pas['pulse_delay_us'] > 4000]


def pulse_width_pas_to_rpm(pulse_width):   
    return 1000000/pulse_width/N_PAS_MAGNETS

df_pas['rpm'] = df_pas.apply(lambda x: pulse_width_pas_to_rpm(x['pulse_delay_us']), axis=1)

df_pas.head()

In [ ]:
mask = df["sensor"] == 'motor_speed'
df_ms = df[mask]

df_ms.rename(columns={2: 'pulse_delay_us',
                      }, inplace=True)
df_ms.dropna(axis=1, how='all',inplace=True)

df_ms = df_ms[df_ms['pulse_delay_us'] > 60000]


def pulse_width_to_rpm(pulse_width):   
    return 1000000/pulse_width

df_ms['rpm'] = df_ms.apply(lambda x: pulse_width_to_rpm(x['pulse_delay_us']), axis=1)

time_delta = df_ms["Datetime"].diff().dt.total_seconds().fillna(0)
df_ms['motor_acceleration'] = df_ms["rpm"].diff()/time_delta


df_ms.head()

In [ ]:
mask = df["sensor"] == 'ina226'
df_ina = df[mask]

df_ina.rename(columns={2: 'INA226 ID',
                       3: 'Battery Voltage',
                       4: 'V_shunt',
                       5: 'Current',
                       6: 'Power',
                      }, inplace=True)
df_ina.dropna(axis=1, how='all',inplace=True)

df_ina.head()

In [ ]:
mask = df["sensor"] == 'baro'
df_baro = df[mask]

df_baro.rename(columns={2: 'temperature[degreeC]',
                       3: 'Pressure[mbar]',
                       4: 'humidity[rh%]',
                      }, inplace=True)

#qnh = pressure at sea level where the readings are being taken.  
qnh=1032.57

def get_altitude(pressure,temperature):
    # The temperature should be the outdoor temperature. 
    # Use the manual_temperature variable if temperature adjustments are required.
    altitude = ((pow((qnh / pressure), (1.0 / 5.257)) - 1) * (temperature + 273.15)) / 0.0065
    return altitude

df_baro['Baro_Altitude'] = df_baro.apply(lambda x: get_altitude(x['Pressure[mbar]'], x['temperature[degreeC]']), axis=1)

df_baro.dropna(axis=1, how='all',inplace=True)

df_baro.head()

In [ ]:
dataframes = {"GPS":df_gps,"Battery_Parameters":df_ina,"Environmental":df_baro}

for df in dataframes:
    dataframes[df].to_csv(df+".csv")


In [ ]:
# Display GPS positions
fig = px.scatter_mapbox(df_gps,
                        lat="latitude",
                        lon="longitude",
                        color="GPS Speed [km/h]",#"slope",#"LOCATION Altitude ( m)",,#"Speed(km/h)", # "abs_acceleration" or "gps_acceleration" or "power"
                        zoom=14,
                        hover_data=["Datetime", "altitude","sats"],
                        #size="LOCATION Accuracy ( m)"
                       )



fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.write_html("output/plot_accurate_gps.html")
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_ina["Datetime"],
    y=df_ina["Battery Voltage[V]"],
    name="Battery Voltage[V]",
    hoverinfo='y',
    line=dict(color="#1f77b4"),
))



fig.add_trace(go.Scatter(
    x=df_gps["Datetime"],
    y=df_gps["GPS Speed [km/h]"],
    name="GPS Speed [km/h]",
    line=dict(color="#ff7f0e"),
    hoverinfo='y',
    yaxis="y2"
))

fig.add_trace(go.Scatter(
    x=df_gps["Datetime"],
    y=df_gps["acceleration"],
    name="gps_acceleration(km/h^2)",
    hoverinfo='y',
    line=dict(color="#d62728"),
    yaxis="y3"
))

fig.add_trace(go.Scatter(
    x=df_ina["Datetime"],
    y=df_ina["Power[mW]"],
    name="Power[mW]",
    hoverinfo='y',
    line=dict(color="#9467bd"),
    yaxis="y4"
))


# Create axis objects
fig.update_layout(
    yaxis=dict(
        title="Battery Voltage[V]",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="GPS Speed [km/h]",
        titlefont=dict(
            color="#ff7f0e"
        ),
        tickfont=dict(
            color="#ff7f0e"
        ),
        anchor="free",
        overlaying="y",
        side="left",
        position=0.05
    ),
    yaxis3=dict(
        title="gps_acceleration(km/h^2)",
        titlefont=dict(
            color="#d62728"
        ),
        tickfont=dict(
            color="#d62728"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis4=dict(
        title="Power[mW]",
        titlefont=dict(
            color="#9467bd"
        ),
        tickfont=dict(
            color="#9467bd"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=0.95
    )
)

# Update layout properties
fig.update_layout(
    title_text="Power, GPS speed, GPS acceleration and battery voltage",
)

fig.update_layout(hovermode="x")
#fig.update_layout(hovermode="x unified")

fig.write_html("output/power_gps_speed.html")


fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_ina["Datetime"],
    y=df_ina["Battery Voltage[V]"],
    name="Battery Voltage[V]",
    hoverinfo='y',
    line=dict(color="#1f77b4"),
))



fig.add_trace(go.Scatter(
    x=df_ina["Datetime"],
    y=df_ina["Current[mA]"],
    name="Current[mA]",
    line=dict(color="#ff7f0e"),
    hoverinfo='y',
    yaxis="y2"
))

fig.add_trace(go.Scatter(
    x=df_ina["Datetime"],
    y=df_ina["V_shunt[mV]"],
    name="V_shunt[mV]",
    hoverinfo='y',
    line=dict(color="#d62728"),
    yaxis="y3"
))

fig.add_trace(go.Scatter(
    x=df_ina["Datetime"],
    y=df_ina["Power[mW]"],
    name="Power[mW]",
    hoverinfo='y',
    line=dict(color="#9467bd"),
    yaxis="y4"
))


# Create axis objects
fig.update_layout(
    yaxis=dict(
        title="Battery Voltage[V]",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="Current[mA]",
        titlefont=dict(
            color="#ff7f0e"
        ),
        tickfont=dict(
            color="#ff7f0e"
        ),
        anchor="free",
        overlaying="y",
        side="left",
        position=0.05
    ),
    yaxis3=dict(
        title="V_shunt[mV]",
        titlefont=dict(
            color="#d62728"
        ),
        tickfont=dict(
            color="#d62728"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis4=dict(
        title="Power[mW]",
        titlefont=dict(
            color="#9467bd"
        ),
        tickfont=dict(
            color="#9467bd"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=0.95
    )
)

# Update layout properties
fig.update_layout(
    title_text="Power, shunt voltage drop, battery voltage and current",
)

fig.update_layout(hovermode="x")
#fig.update_layout(hovermode="x unified")

fig.write_html("output/power_params.html")


fig.show()

## Display all variables

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=8, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.01)

fig.update_layout(hovermode="x unified")
 

# fig = make_subplots(rows=2, cols=1,
#                     specs=[[{'type': 'xy'}],
#                            [{'type': 'mapbox'}]])

fig.add_trace(go.Scatter(
    x=df_ina["Datetime"],
    y=df_ina["Power[mW]"],
    name="Power[mW]",
    hoverinfo='y'),
    row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_ina["Datetime"],
    y=df_ina["Battery Voltage[V]"],
    name="Battery Voltage[V]",
    hoverinfo='y'),
    row=2, col=1)
             

fig.add_trace(go.Scatter(
    
    x=df_gps["Datetime"],
    y=df_gps["GPS Speed [km/h]"],
    name="GPS Speed [km/h]",
    hoverinfo='y'),
    row=3, col=1)

fig.add_trace(go.Scatter(
    
    x=df_ms["Datetime"],
    y=df_ms["motor_acceleration"],
    name="Motor Acceleration(km/h^2)",
    hoverinfo='y'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    
    x=df_gps["Datetime"],
    y=df_gps["altitude"],
    name="GPS Altitude[m]",
    hoverinfo='y'),
    row=5, col=1)

fig.add_trace(go.Scatter(
    
    x=df_baro["Datetime"],
    y=df_baro["Baro_Altitude"],
    name="Baro_Altitude[m]",
    hoverinfo='y'),
    row=6, col=1)


fig.add_trace(go.Scatter(
    x=df_ms["Datetime"],
    y=df_ms["rpm"],
    name="Motor RPM",
    hoverinfo='y'),
    row=7, col=1)

fig.add_trace(go.Scatter(
    x=df_pas["Datetime"],
    y=df_pas["rpm"],
    name="Pedal Assist Sensor RPM",
    hoverinfo='y'),
    row=8, col=1)
              






fig.update_layout(title_text="Battery Power, Voltage, GPS speed and GPS Acceleration, GPS Altitude")

fig.write_html("output/gps_data_ina_baro.html")

fig.show()

## Display the interesting variables variables

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=5, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.01)

fig.update_layout(hovermode="x unified")
 

# fig = make_subplots(rows=2, cols=1,
#                     specs=[[{'type': 'xy'}],
#                            [{'type': 'mapbox'}]])

fig.add_trace(go.Scatter(
    x=df_ina["Datetime"],
    y=df_ina["Power"],
    name="Power[mW]",
    hoverinfo='y'),
    row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_ina["Datetime"],
    y=df_ina["Battery Voltage"],
    name="Battery Voltage[V]",
    hoverinfo='y'),
    row=2, col=1)
             

fig.add_trace(go.Scatter(
    
    x=df_baro["Datetime"],
    y=df_baro["Baro_Altitude"],
    name="Baro_Altitude[m]",
    hoverinfo='y'),
    row=3, col=1)


fig.add_trace(go.Scatter(
    x=df_ms["Datetime"],
    y=df_ms["rpm"],
    name="Motor RPM",
    hoverinfo='y'),
    row=4, col=1)

fig.add_trace(go.Scatter(
    x=df_pas["Datetime"],
    y=df_pas["rpm"],
    name="Pedal Assist Sensor RPM",
    hoverinfo='y'),
    row=5, col=1)
              






fig.update_layout(title_text="Battery Power, Voltage, Barometric Altitude, Motor RPM, Pedal Assist Sensor RPM")

fig.write_html("output/gps_data_ina_baro.html")

fig.show()

# Show long,lat,altitude and Speed in single 3D plot

## find nearest barometer value

In [ ]:
# Run only once
df_gps = pd.merge_asof(df_gps,df_baro , on = 'Datetime', direction = 'nearest')
df_gps = pd.merge_asof(df_gps,df_ina , on = 'Datetime', direction = 'nearest')
df_gps.columns.tolist()

In [ ]:
import plotly.graph_objects as go


fig = go.Figure(data=go.Scatter3d(
    x=df_gps["longitude"],
    y=df_gps["latitude"],
    z=df_gps["altitude"],
    marker=dict(
        size=4,
        color = "blue"
    ),
    name='GPS altitude'
))


fig2 = go.Figure(data=go.Scatter3d(
    x=df_gps["longitude"],
    y=df_gps["latitude"],
    z=df_gps["Baro_Altitude"],
    marker=dict(
        size=4,
        color = "red"
    ),
    name='Barometer altitude'

))

fig.add_trace(fig2.data[0]) # adds the line trace to the first figure



fig.update_layout(
     width=800,
#     height=700,
#     autosize=False,
    scene=dict(
        xaxis = dict(title='Longitude'),
        yaxis = dict(title='Latitude'),
        zaxis = dict(title='Altitude[m]'),
#         camera=dict(
#             up=dict(
#                 x=0,
#                 y=0,
#                 z=1
#             ),
#             eye=dict(
#                 x=0,
#                 y=1.0707,
#                 z=1,
#             ),
#         ),
#         aspectratio = dict( x=1, y=1, z=0.7 ),
#         aspectmode = 'manual'
    ),
)

fig.write_html("output/gps_vs_barometer_altitude_3d.html")
fig.show()


In [ ]:
import plotly.graph_objects as go


fig = go.Figure(data=go.Scatter3d(
    x=df_gps["longitude"],
    y=df_gps["latitude"],
    z=df_gps["Baro_Altitude"],
    marker=dict(
        size=4,
        color=df_gps["Power[mW]"],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(
            title="Power[mW]"
        )

    ),
    text = 'Power[mW]:' +df_gps["Power[mW]"].astype(str),
    line=dict(
        color='darkblue',
        width=2
    ),
))

fig.update_layout(
    width=800,
    #height=700,
    #autosize=False,
    scene=dict(
        xaxis = dict(title='Longitude'),
        yaxis = dict(title='Latitude'),
        zaxis = dict(title='Barometric Altitude[m]'),
#         camera=dict(
#             up=dict(
#                 x=0,
#                 y=0,
#                 z=1
#             ),
#             eye=dict(
#                 x=0,
#                 y=1.0707,
#                 z=1,
#             ),
#         ),
#         aspectratio = dict( x=1, y=1, z=0.7 ),
#         aspectmode = 'manual'
    ),
)




fig.write_html("output/gps_speed_3d.html")
fig.show()


# Make KML data of gps/baro data

In [ ]:
import simplekml
kml = simplekml.Kml()

In [ ]:
pts = df_gps[['longitude', 'latitude','Baro_Altitude']].to_records(index=False).tolist()

linestring = kml.newlinestring(name="Pathway", description="A pathway in Kirstenbosch")
linestring.coords = pts
linestring.altitudemode = simplekml.AltitudeMode.absolute
linestring.extrude = 1

In [ ]:
kml.save(path = "output/data.kml")

# Do machine learning

In [ ]:
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 

In [ ]:
 
boston = load_boston()
x, y = boston.data, boston.target
x

In [ ]:
x, y = df_ina[["Battery Voltage","Current","V_shunt"]], df_ina["Power"]
x

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.15)

xgbr = xgb.XGBRegressor(verbosity=0)
print(xgbr)

xgbr.fit(xtrain, ytrain)

 

score = xgbr.score(xtrain, ytrain)   

print("Training score: ", score) 

 

# - cross validataion 
scores = cross_val_score(xgbr, xtrain, ytrain, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())

kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbr, xtrain, ytrain, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

 

ypred = xgbr.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

In [ ]:
%matplotlib qt
x_ax = range(len(ytest))
plt.scatter(x_ax, ytest, s=5, color="blue", label="original")
plt.plot(x_ax, ypred, lw=0.8, color="red", label="predicted")
plt.legend()
plt.show()

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


fig = px.scatter(x=x_ax, y=ytest, opacity=0.65)
fig.add_traces(go.Scatter(x=x_ax, y=ypred, name='Regression Fit'))
fig.show()